In [1]:
# Instalar bibliotecas necessárias
!pip install tiktoken torch

# Importar bibliotecas essenciais
import os
import random
import json
import torch
import sys

# Adicionar caminho do módulo BigramModel
sys.path.append(r"C:\Users\secad\Downloads")  
from bigram_model import BigramModel

# Diretório do corpus
corpus_dir = r"C:\Users\secad\Downloads\corpus"

# Função para dividir o corpus em treino e teste
def split_corpus(directory, train_ratio=0.8):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(".json")]
    random.shuffle(files)
    split_index = int(len(files) * train_ratio)
    return files[:split_index], files[split_index:]

# Função para filtrar tokens raros
def filter_rare_tokens(text, min_frequency=2):
    tokens = text.split()
    token_counts = {token: tokens.count(token) for token in set(tokens)}
    return " ".join([token for token in tokens if token_counts[token] >= min_frequency])

# Função para extrair texto dos arquivos JSON
def extract_text(files):
    combined_text = ""
    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
            combined_text += data.get("text", "") + " "
    return combined_text.strip()

# Divisão do corpus
print("Dividindo o corpus em treino e teste...")
train_files, test_files = split_corpus(corpus_dir)
print(f"{len(train_files)} arquivos para treino e {len(test_files)} arquivos para teste.")

# Extração de texto do treino e teste
print("Extraindo texto do conjunto de treino e teste...")
train_text = extract_text(train_files)
test_text = extract_text(test_files)

# Balancear o treino com uma parte do teste
portion_of_test_to_train = " ".join(test_text.split()[:10000])  # Pegue 10.000 palavras do teste
train_text += " " + portion_of_test_to_train

# Filtrar tokens raros do treino
train_text = filter_rare_tokens(train_text)

print(f"Conjunto de treino final: {len(train_text.split())} palavras.")
print(f"Conjunto de teste: {len(test_text.split())} palavras.")

# Treinamento do modelo Bigram
print("Treinando o modelo Bigram...")
bigram = BigramModel(max_tokens=100000)  # Limite de tokens ajustado
bigram.train(train_text)
print("Treinamento concluído.")

# Salvar modelo
print("Salvando o modelo treinado...")
bigram.save_model("bigram_model.pth")
print("Modelo salvo como 'bigram_model.pth'.")

# Carregar modelo
print("Carregando modelo treinado...")
loaded_bigram = BigramModel.load_model("bigram_model.pth")
print("Modelo carregado com sucesso.")

# Gerar texto aleatório
print("\nGerando texto aleatório:")
print(loaded_bigram.generate_text(50))

# Gerar texto com seed
print("\nGerando texto a partir de seed:")
print(loaded_bigram.generate_text(50, seed=42))

# Calcular perplexidade no conjunto de teste
print("\nCalculando perplexidade do conjunto de teste...")
perplexity = loaded_bigram.perplexity(test_text[:500])  # Avalia os primeiros 500 tokens para teste
print(f"Perplexidade do conjunto de teste: {perplexity}")

   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   ---------------------------------------- 883.8/883.8 kB 6.6 MB/s eta 0:00:00
Dividindo o corpus em treino e teste...
1765 arquivos para treino e 442 arquivos para teste.
Extraindo texto do conjunto de treino e teste...
Conjunto de treino final: 2754836 palavras.
Conjunto de teste: 664729 palavras.
Treinando o modelo Bigram...
Treinamento concluído.
Salvando o modelo treinado...
Modelo salvo como 'bigram_model.pth'.
Carregando modelo treinado...


C:\Users\secad\Downloads\bigram_model.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Modelo carregado com sucesso.

Gerando texto aleatório:
682 1979 — Helge escritor brasileiro (2013). * Bandeira, uma linguagem cromática e menos 719 4 **Mona atriz estado-unidenseamentos ==Demografia== População de Vagos e político presidente eleito Papa

Gerando texto a partir de seed:
 multicel branco, pertenciam atirado em Paris. Ele completou um referencial pela combinação, o povoação da Ponte M % Várias possíveis no Museu Nacional de teologia === Aéficas em 0 1921, Feira é

Calculando perplexidade do conjunto de teste...
Perplexidade do conjunto de teste: 14402682.766790349
